In [1]:
import pandas as pd

stats_2023 = pd.read_csv("../../data/preprocess/pre-processed.csv")
stats_2023.dtypes

Field ID           int64
Field Name        object
Crop ID            int64
Crop Name         object
Crop Type         object
Planting Area    float64
Season ID          int64
Season            object
Field Type        object
Field Area       float64
Per Yield        float64
Per Cost         float64
Per Price        float64
Yield            float64
Cost             float64
Selling          float64
Revenue          float64
Profit           float64
dtype: object

# 初始化

In [2]:
import pulp
import numpy as np

# 定义辅助变量

In [3]:
fields_id = list(map(int, stats_2023["Field ID"].unique()))  # 所有的地块
crops_id = list(map(int, stats_2023["Crop ID"].unique()))  # 所有的作物
seasons_id = list(map(int, stats_2023["Season ID"].unique()))  # 所有的时节
fields_id.sort()
crops_id.sort()
seasons_id.sort()

years = range(2023, 2031)  # 要优化的年份


def pop_ndarray(arr, element_to_pop):
    index = np.where(arr == element_to_pop)[0]
    if index.size > 0:
        arr = np.delete(arr, index)
    return arr


# A类粮食作物的集合
grains_A = stats_2023[
    (stats_2023["Crop Type"] == "粮食") | (stats_2023["Crop Type"] == "粮食（豆类）")
]["Crop ID"].unique()
grains_A = pop_ndarray(grains_A, 16)
grains_A.sort()

# B类粮食作物的集合
grains_B = np.array([16])

# A类蔬菜作物的集合
vege_A = stats_2023[
    (stats_2023["Crop Type"] == "蔬菜") | (stats_2023["Crop Type"] == "蔬菜（豆类）")
]["Crop ID"].unique()
for i in range(3):
    vege_A = pop_ndarray(vege_A, 35 + i)
vege_A.sort()

# B类蔬菜作物的集合
vege_B = np.array([35, 36, 37])

# 食用菌作物的集合
mush = stats_2023[stats_2023["Crop Type"] == "食用菌"]["Crop ID"].unique()
mush.sort()

# 豆类作物的集合
beans = stats_2023[
    (stats_2023["Crop Type"] == "粮食（豆类）")
    | (stats_2023["Crop Type"] == "蔬菜（豆类）")
]["Crop ID"].unique()
beans.sort()

# 第 i 个地块的类型（如平旱地、梯田、山坡地、智能大棚、普通大棚、水浇地）
t_i = {
    i: stats_2023[stats_2023["Field ID"] == i]["Field Type"].values[0]
    for i in fields_id
}

# 第 i 个地块在第 s 季可种植的作物集合
T_hat_i_s: dict[tuple : np.ndarray] = {}


for i in fields_id:
    if t_i[i] in ["平旱地", "梯田", "山坡地"]:
        T_hat_i_s[(i, 1)] = grains_A
        T_hat_i_s[(i, 2)] = np.array([])
    elif t_i[i] in ["水浇地"]:
        T_hat_i_s[(i, 1)] = np.concatenate([grains_B, vege_A])
        T_hat_i_s[(i, 2)] = vege_B
    elif t_i[i] in ["普通大棚"]:
        T_hat_i_s[(i, 1)] = vege_A
        T_hat_i_s[(i, 2)] = mush
    elif t_i[i] in ["智慧大棚"]:
        T_hat_i_s[(i, 1)] = vege_A
        T_hat_i_s[(i, 2)] = vege_A

S: dict[tuple : np.int64] = {}  # 期望产量
Y: dict[tuple : np.int64] = {}  # 单位面积产量
P: dict[tuple : np.int64] = {}  # 售价
C: dict[tuple : np.int64] = {}  # 单位面积成本

for j in crops_id:
    # 所有参数与 2023 年的保持一致
    for k in years:
        S[(j, k)] = stats_2023.groupby(["Crop ID"])["Selling"].agg("sum").values[j - 1]
        Y[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Yield"].values[0]
        P[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Price"].values[0]
        C[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Cost"].values[0]

# 定义决策变量

In [4]:
# A^1_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为不为大棚地块则为 0
# A^2_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为大棚地块则为 0
# A_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积
A_ijks = pulp.LpVariable.dicts(
    "A",
    [
        (i, j, k, s)
        for i in fields_id
        for j in crops_id
        for k in years
        for s in seasons_id
    ],
    lowBound=0,
    cat=pulp.LpContinuous,
)


# Rename
for i in fields_id:
    for j in crops_id:
        for k in years:
            for s in seasons_id:
                A_ijks[(i, j, k, s)].name = f"A_{i}_{j}_{k}_{s}"


# A_{ijk} = \sum_s A^2_{ijks} + A^1_{ij(k-1)2} + A^1_{ijks}
A_ijk: dict[tuple : pulp.LpVariable] = {}
for i in fields_id:
    for j in crops_id:
        for k in years:
            if k == 2023:
                continue
            A_ijk[(i, j, k)] = pulp.lpSum([A_ijks[i, j, k, s] for s in seasons_id])


# 2023 年的 A_{ijks} 已知
for i in fields_id:
    for j in crops_id:
        for s in seasons_id:
            matching_rows = stats_2023[
                (stats_2023["Field ID"] == i)
                & (stats_2023["Crop ID"] == j)
                & (stats_2023["Season ID"] == s)
            ]

            area = (
                matching_rows["Planting Area"].values[0]
                if not matching_rows.empty
                else 0
            )

            A_ijks[(i, j, 2023, s)].lowBound = area
            A_ijks[(i, j, 2023, s)].upBound = area

# 2023 年的 A_{ijk} 已知
for i in fields_id:
    for j in crops_id:
        A_ijk[(i, j, 2023)] = pulp.lpSum([A_ijks[i, j, 2023, s] for s in seasons_id])

B_ijks = pulp.LpVariable.dicts(
    "B",
    [
        (i, j, k, s)
        for i in fields_id
        for j in crops_id
        for k in years
        for s in seasons_id
    ],
    lowBound=0,
    cat=pulp.LpBinary,
)

A_star = {}  # 地块面积

for i in fields_id:
    A_star[i] = stats_2023[stats_2023["Field ID"] == i]["Field Area"].values[0]

# 定义目标函数

In [5]:
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)

alpha = 0.5  # 情况二

# 定义辅助变量 Z_ijk
Z_ijk = pulp.LpVariable.dicts(
    "Z",
    [(i, j, k) for i in fields_id for j in crops_id for k in years],
    lowBound=0,
    cat=pulp.LpContinuous,
)

# 线性化的目标函数
L = pulp.lpSum(
    [
        (
            Y[(j, k)] * A_ijk[(i, j, k)] * P[(j, k)]
            - C[(j, k)] * A_ijk[(i, j, k)]
            - alpha * Z_ijk[(i, j, k)] * P[(j, k)]
        )
        for i in fields_id
        for j in crops_id
        for k in years
    ]
)

# 设置目标函数
prob += L

In [6]:
# 辅助约束： Z_ijk >= Y_jk * A_ijk - S_jk
for i in fields_id:
    for j in crops_id:
        for k in years:
            prob += Z_ijk[(i, j, k)] >= Y[(j, k)] * A_ijk[(i, j, k)] - S[(j, k)]

# 添加约束 Z_ijk >= 0 已经通过 LpVariable 的 lowBound=0 实现

In [7]:
# 辅助约束
M1 = 10000  # 一个很大的数

for i in fields_id:
    for j in crops_id:
        for k in years:
            for s in seasons_id:
                prob += A_ijks[(i, j, k, s)] <= B_ijks[(i, j, k, s)] * M1

In [8]:
# 约束：一个地块上每种作物合起来的种植面积不超过相应地块的总面积
for i in fields_id:
    for s in seasons_id:
        for k in years:
            prob += (
                pulp.lpSum([A_ijks[(i, j, k, s)] for j in T_hat_i_s[(i, s)]])
                <= A_star[i]
            )
            for j in crops_id:
                if j not in T_hat_i_s[(i, s)]:
                    prob += A_ijks[(i, j, k, s)] == 0


In [9]:
# 约束：不能重茬种植
for i in fields_id:
    for j in crops_id:
        for k in years:
            if k >= years[-1]:  # 最后一年不需要考虑后面的年份
                continue

            # 如果作物 j 在第 i 块地的第 1 季度被种植
            if j in T_hat_i_s[(i, 1)] and j not in T_hat_i_s[(i, 2)]:
                prob += B_ijks[(i, j, k, 1)] + B_ijks[(i, j, k + 1, 1)] <= 1
            # 如果作物 j 在第 i 块地的第 1 和第 2 季度都被种植
            elif j in T_hat_i_s[(i, 1)] and j in T_hat_i_s[(i, 2)]:
                prob += B_ijks[(i, j, k, 1)] + B_ijks[(i, j, k, 2)] <= 1

In [10]:
# 约束：每个地块（含大棚）的所有土地三年内至少种植一次豆类作物
for i in fields_id:
    for k in years:
        if k >= years[-2]:  # 最后两年不需要考虑后面的年份
            continue

        prob += (
            pulp.lpSum(
                [
                    B_ijks[(i, j, kk, s)]
                    for kk in range(k, k + 3)
                    for s in seasons_id
                    for j in beans
                    if j in T_hat_i_s[i, s]
                ]
            )
            >= 1
        )

        for j in beans:
            for s in seasons_id:
                if j in T_hat_i_s[i, s]:
                    # 如果种植，则种植整个地块面积
                    prob += A_ijks[(i, j, k, s)] == A_star[i] * B_ijks[i, j, k, s]

In [11]:
# 约束：第一季种了水稻的水浇地，第二季不能种 B 类蔬菜
for i in fields_id:
    for k in years:
        if t_i[i] == "水浇地":
            for j1 in grains_B:
                for j2 in vege_B:
                    prob += B_ijks[(i, j1, k, 1)] + B_ijks[(i, j2, k, 2)] <= 1

In [12]:
# 约束：第一季种了 A 类蔬菜的水浇地，第二季的 B 类蔬菜不能混种在一个地块上
for i in fields_id:
    for k in years:
        if t_i[i] == "水浇地":
            for j2 in vege_B:
                prob += A_ijks[(i, j2, k, 2)] == A_star[i] * B_ijks[i, j2, k, 2]

In [13]:
M2 = 0.3

# 约束：单块地，单种的种植面积不能过小
for i in fields_id:
    for k in years:
        for s in seasons_id:
            for j in T_hat_i_s[(i, s)]:
                prob += A_ijks[(i, j, k, s)] >= M2 * A_star[i] * B_ijks[(i, j, k, s)]

# 模型求解

In [14]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/zivmax/CUMCM_2024/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e8aefc291aea49e4a80951b9008453be-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/e8aefc291aea49e4a80951b9008453be-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 97807 COLUMNS
At line 418368 RHS
At line 516171 BOUNDS
At line 556024 ENDATA
Problem MODEL has 97802 rows, 88560 columns and 196576 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.03257e+08 - 0.08 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 6396 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2212 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1934 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1098 strengthened rows, 0 substituti

# 整理结果

In [ ]:
A_ijks_output = {}

for v in prob.variables():
    if v.name.startswith("A_star"):
        continue

    if v.name.startswith("A"):
        name = v.name.replace("A_", "")
        A_ijks_output[tuple(map(int, name.split("_")))] = v.varValue


A_ijks_output = dict(
    sorted(
        A_ijks_output.items(),
        key=lambda item: (
            int(item[0][2]),  # year
            int(item[0][3]),  # season
            int(item[0][0]),  # field
            int(item[0][1]),  # crop
        ),
    )
)

for key, value in A_ijks_output.items():
    print(key, value)

(1, 1, 2023, 1) 0.0
(1, 2, 2023, 1) 0.0
(1, 3, 2023, 1) 0.0
(1, 4, 2023, 1) 0.0
(1, 5, 2023, 1) 0.0
(1, 6, 2023, 1) 80.0
(1, 7, 2023, 1) 0.0
(1, 8, 2023, 1) 0.0
(1, 9, 2023, 1) 0.0
(1, 10, 2023, 1) 0.0
(1, 11, 2023, 1) 0.0
(1, 12, 2023, 1) 0.0
(1, 13, 2023, 1) 0.0
(1, 14, 2023, 1) 0.0
(1, 15, 2023, 1) 0.0
(1, 16, 2023, 1) 0.0
(1, 17, 2023, 1) 0.0
(1, 18, 2023, 1) 0.0
(1, 19, 2023, 1) 0.0
(1, 20, 2023, 1) 0.0
(1, 21, 2023, 1) 0.0
(1, 22, 2023, 1) 0.0
(1, 23, 2023, 1) 0.0
(1, 24, 2023, 1) 0.0
(1, 25, 2023, 1) 0.0
(1, 26, 2023, 1) 0.0
(1, 27, 2023, 1) 0.0
(1, 28, 2023, 1) 0.0
(1, 29, 2023, 1) 0.0
(1, 30, 2023, 1) 0.0
(1, 31, 2023, 1) 0.0
(1, 32, 2023, 1) 0.0
(1, 33, 2023, 1) 0.0
(1, 34, 2023, 1) 0.0
(1, 35, 2023, 1) 0.0
(1, 36, 2023, 1) 0.0
(1, 37, 2023, 1) 0.0
(1, 38, 2023, 1) 0.0
(1, 39, 2023, 1) 0.0
(1, 40, 2023, 1) 0.0
(1, 41, 2023, 1) 0.0
(2, 1, 2023, 1) 0.0
(2, 2, 2023, 1) 0.0
(2, 3, 2023, 1) 0.0
(2, 4, 2023, 1) 0.0
(2, 5, 2023, 1) 0.0
(2, 6, 2023, 1) 0.0
(2, 7, 2023, 1) 55.0
(2, 8,

In [ ]:
# 将字典转换为 DataFrame
df = pd.DataFrame(
    [(field, crop, year, season, value) for (field, crop, year, season), value in A_ijks_output.items()],
    columns=['Field', 'Crop', 'Year', 'Season', 'Planting']
)

# 计算平均值时需要指定哪些列进行聚合
df = df.groupby(["Field", "Crop", "Season", "Year"]).agg("mean").reset_index()
df = df[~((df["Season"] == 2) & (df["Field"] < 27))]  # 去掉单季地块第二季度的数据

# 获取不包括 2023 年的唯一年份
years = df["Year"].unique()
years = [year for year in years if year != 2023]


# 创建一个 Excel writer 对象
with pd.ExcelWriter(f"part-2.xlsx") as writer:
    for year in years:
        # 过滤特定年份的数据
        df_year = df[df["Year"] == year]

        # 透视表，Field 作为行，Crop 作为列
        pivot_table = df_year.pivot_table(
            index=["Season", "Field"], columns="Crop", values="Planting", aggfunc="sum"
        ).round(
            1
        )  # 保留一位小数

        # 将透视表写入 Excel 的 sheet
        pivot_table.to_excel(writer, sheet_name=str(year))

print(f"Exported to part-2.xlsx")

Exported to alpha-0.0.xlsx
